# Intro

## loading required data and libs

In [1]:
! pip install -U spacy
! pip install -U -r environment/requirenments.txt
! pip install -U sentence-transformers
! pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
! python -m spacy download en_core_web_lg
! python -m spacy download en_core_web_sm
!pip install contextualSpellCheck
!pip install autocorrect
!pip install black

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## importing

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import black
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from autocorrect import Speller
import json
import inspect
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# from sentence_transformers import SentenceTransformer, util
import torch
import tokenize
from tokenize import TokenError
import io
tqdm.pandas()
import torch
from sentence_transformers import SentenceTransformer, util

import nltk
import re
import string

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.pipeline import Pipeline
import sys
import textwrap
import wandb
import spacy
import contextualSpellCheck
from sklearn.preprocessing import FunctionTransformer
ner = spacy.load("en_core_web_lg")
check_spelling = spacy.load("en_core_web_sm")

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('data/')
sys.path.append(str('AI4Code'))
%load_ext autoreload
%autoreload 2
contextualSpellCheck.add_to_pipe(check_spelling)
# black_refactroing_mode = black.FileMode()

## text utils

In [4]:
spell = Speller(fast=True)
def correct_spelling(text):
    if text:
        return spell(text)
    else:
        return text
        

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    if not text:
        return ''
    text = text.lower()
    text = text.strip()
    text = ' '.join(text.split())
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = text.strip()
    return text

def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    combined_text = ' '.join(tokenized_text)
    return combined_text

def clean_code(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.replace('[', ' ').replace(']', ' ').replace('(', ' ')\
    .replace(')', ' ').replace('{', ' ').replace('}', ' ').replace('=', ' ').replace(',', ' ')
    text = text.lower()
    text = text.replace('_', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('.', ' ')
    text = re.sub(r'".*"', ' ', text)
    text = re.sub(r"'.*'", ' ', text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

def code_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    try:
        text = black.format_str(text, mode=black_refactroing_mode) ## maybe remove
    except:
        pass
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_code(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    combined_text = ' '.join(tokenized_text)
    return combined_text

def count_hastags(row):
    "Count the number of hashtags "
    row['hash_count'] = row['source'].count('# ') if row['cell_type']=='markdown' else 0
    return row

## reading utils

In [5]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

def read_train_data(data_dir, NUM_TRAIN = 10000, OFFSET=0):
    def read_notebook(path):
        return (
            pd.read_json(
                path,
                dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)  # final path component
            .rename_axis('cell_id')
        )

    paths_train = list((data_dir / 'train').glob('*.json'))[OFFSET:NUM_TRAIN]
    notebooks_train = [
      read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
    ]
    df = (
      pd.concat(notebooks_train)
      .set_index('id', append=True)
      .swaplevel()
      .sort_index(level='id', sort_remaining=False)
    )
    return df

def get_df_orders_and_ranks(df, data_dir):
    # train orders
    df_orders = pd.read_csv(
      data_dir / 'train_orders.csv',
      index_col='id',
      squeeze=True,
    ).str.split()  # cell_ids str -> list


    df_orders_ = df_orders.to_frame().join(
      # reset only one index out of many -> "cell_id"; make a list out of cells in train data
      df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
      how='right',
    )

    ranks = {}
    for id_, cell_order, cell_id in df_orders_.itertuples():
        ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

    df_ranks = (
      pd.DataFrame
      .from_dict(ranks, orient='index')
      .rename_axis('id')
      .apply(pd.Series.explode)
      .set_index('cell_id', append=True)
    )
    # now we have
    # id cell_id rank
    return df_orders, df_ranks


def get_ancestors(data_dir, ids):
    # Split, keeping notebooks with a common origin (ancestor_id) together
    df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
    return df_ancestors.loc[ids, 'ancestor_id']

## Reading

In [6]:
# example pipeline work
df = read_train_data(data_dir, NUM_TRAIN=10000)
df_orders, df_ranks = get_df_orders_and_ranks(df, data_dir)

print(f"Df shape is {df.shape}")


Df shape is (461582, 2)


# Feature Engineering

In [7]:
# general columns
def prepare_data(df, func, new_column, on_column='source'):
    df[new_column] = df.progress_apply(lambda x: func(x[on_column]), axis=1)
    return df

## funcs

In [8]:
## lemmatizer 
lem = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize_python_code_names(text):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(text).readline)
        strings = [tok.string for tok in code_text if tok.type==1]
    except Exception:
        return None # Error happened
    return ' '.join(strings)

def tokenize_python_code_comments(text):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(text).readline)
        strings = [tok.string for tok in code_text if tok.type==55]
    except Exception:
        return None # Error happened
    return ' '.join(strings)

def lemm_sentence(text):
    lst_txt = [lem.lemmatize(word) for word in text.split()]
    return ' '.join(lst_txt)

def collect_entities(text):
    "Named Entity Recognition"
    rs = ner(text)
    labels = []
    for r in rs.ents:
        labels.append(r.label_)
    return ' '.join(labels)

def convert_to_TfidfVector(df, max_features=100):
    tfidf = TfidfVectorizer(min_df=0.01, max_features=max_features)
    return tfidf, tfidf.fit_transform(df.astype(str))

## ALPHA 1 todo list


## Machine Learning glossary feature

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
ml_glossary = pd.read_csv("machine_learning_glossary_terms.csv")
# sentences we will be searching through
corpus = np.array(ml_glossary['definition'])
terms = np.array(ml_glossary['term'])
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [10]:
# better do not vector but words and the put them to TFID
top_k=6
def get_top_glossary_terms(query):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k, query_chunk_size=1000)
    hits = ['_'.join(str(terms[hit['corpus_id']]).split()) for hit in hits[0]]
    return ' '.join(hits)

## markdowns pipeline

In [11]:
markdowns_pipeline = Pipeline([
    ('Clearing and parsing source', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': text_preprocessing,
             'new_column': "source_clean",
             'on_column': "source"})),
    ('Spelling', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': correct_spelling,
             'new_column': "source_clean",
             'on_column': "source_clean"})),
    ('Entities', FunctionTransformer(
         func=prepare_data,
         kw_args={
         'func': collect_entities,
         'new_column': "entities",
         'on_column': "source_clean"})),
    ('ML glossary feature', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': get_top_glossary_terms,
             'new_column': "glossary_ml_terms",
             'on_column': "source_clean"})),
    ('Lemmatizing sentences', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lemm_sentence,
             'new_column': "source_clean",
             'on_column': "source_clean"})),
], verbose=True)

## Code pipeline

In [12]:
comments_sub_pipeline = Pipeline([
    ('[comments] Extracting code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': tokenize_python_code_comments,
             'new_column': "code_comments",
             'on_column': "source"})),
    ('[comments] Clearing and parsing code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': text_preprocessing,
             'new_column': "code_comments",
             'on_column': "code_comments"})),
    ('[comments] Correcting spelling', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': correct_spelling,
             'new_column': "code_comments",
             'on_column': "code_comments"})),
    ('[comments] Lemmatizing code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lemm_sentence, # todo maybe make additional pipeline for comments and multisteps pipe
             'new_column': "code_comments",
             'on_column': "code_comments"})),
        # todo make comments statistics and implement it if column is not empty
    ('[3] Comments stats', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lambda x: len(str(x).split()),
             'new_column': "len_of_code_comments",
             'on_column': "code_comments"})),
], verbose=True)

In [13]:
code_pipeline = Pipeline([
    ('Clearing and parsing source', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': code_preprocessing,
             'new_column': "source_clean",  # TODO maybe even remove it or leave the same. check on gridsearch later
             'on_column': "source"})),
    
    ('Extacting code names', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': tokenize_python_code_names,
             'new_column': "python_code_names",
             'on_column': "source"})),
    
    ('Comments', comments_sub_pipeline)
], verbose=True)

In [14]:
# counting features
stats_pipeline = Pipeline([
    ('Length', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lambda x: len(str(x).split()),
             'new_column': "text_len",
             'on_column': "source_clean"})),
    ('Word count', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lambda x: len(str(x)),
             'new_column': "text_word_count",
             'on_column': "source_clean"})),

], verbose=True)

## connecting together

In [15]:
def preprocess_data(df):
    df['index_col'] = range(1, len(df) + 1)
    
    # splitting on markdown and code
    markdowns = df[df['cell_type'] == 'markdown'].reset_index()
    codes = df[df['cell_type'] == 'code'].reset_index()

    print(f"Df shape is {df.shape} markdowns {markdowns.shape} code {codes.shape}")
    
    markdowns = markdowns_pipeline.fit_transform(markdowns)
    markdowns = stats_pipeline.fit_transform(markdowns)
    codes = code_pipeline.fit_transform(codes)
    codes = stats_pipeline.fit_transform(codes)
    
    # setting rank
    codes['rank'] = codes.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1
    markdowns['rank'] = 0

    # Stacking back together
    df = pd.concat([codes, markdowns]).set_index(['id', 'cell_id', 'index_col'])
    
    return df

# Running

In [ ]:
df = preprocess_data(df)

Df shape is (461582, 3) markdowns (156444, 5) code (305138, 5)



[Pipeline]  (step 1 of 5) Processing Clearing and parsing source, total=  11.9s



[Pipeline] .......... (step 2 of 5) Processing Spelling, total= 1.2min


# Training

## splitting

In [ ]:
NVALID = 0.1

ids = df.index.unique('id')
ancestors = get_ancestors(data_dir, ids)
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors)) 
ids_train, ids_valid = ids[ids_train], ids[ids_valid]
df_train = df.loc[ids_train, :].sort_index(level='index_col')
df_valid = df.loc[ids_valid, :].sort_index(level='index_col')

In [ ]:
TFID_FEATURES = {
    'entities': None,
    "glossary_ml_terms": None,
    "python_code_names": None,
    "code_comments": None,
    "source_clean": None,
} # todo gridsearch

In [ ]:
tfid_transformers = {}

In [ ]:
X_train = np.array([])
for feature, max_n in tqdm(TFID_FEATURES.items()):
    tfid_tr, tfid_vector = convert_to_TfidfVector(df_train[feature], max_n)
    tfid_transformers[feature] = tfid_tr
    if not X_train.shape[0]:
        X_train = tfid_vector
    else:
        X_train = sparse.hstack((X_train, tfid_vector))

X_train = sparse.hstack((
    X_train, 
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
    ))

In [ ]:
# Rank of each cell within the notebook
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

In [ ]:
from xgboost import XGBRanker

model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)
model.fit(X_train, y_train, group=groups)

In [ ]:
tfid_transformers['entities']

In [ ]:
X_valid = np.array([])
for feature, _ in tqdm(TFID_FEATURES.items()):
    transformer = tfid_transformers[feature]
    tfid_vector = transformer.transform(df_valid[feature].astype('str'))
    if not X_valid.shape[0]:
        X_valid = tfid_vector
    else:
        X_valid = sparse.hstack((X_valid, tfid_vector))

X_valid = sparse.hstack((
    X_valid, 
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
    ))

In [ ]:
X_valid.shape

In [ ]:
X_train.shape

In [ ]:
y_valid = df_orders.loc[ids_valid]

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)

In [ ]:
from bisect import bisect


def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max


In [ ]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy)

In [ ]:
kendall_tau(y_valid, y_pred)

# Saving

In [186]:
sparse.save_npz('data/df_tfisd_15k.npz', df_tfisd)

In [123]:
sparse_matrix = sparse.load_npz('data/df_tfisd_15k.npz')